In [ ]:
!pip install geopandas shapely fiona pyproj

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# CHANGE input_folder: Path to the folder in your Google Drive containing your vector files
# Example: 'My Drive/Shapefiles'
input_folder = "/content/drive/My Drive/Shapefiles"   # @param {type:"string"} # Folder with input vector files
target_crs = "EPSG:4326"  # @param {type:"string"} # Target CRS (Web Mercator by default)
output_folder = os.path.join(input_folder, "Shapefiles_Reprojected")  # @param {type:"string"} # Folder to save
os.makedirs(output_folder, exist_ok=True)
print(f"Reprojected shapefiles will be saved in: {output_folder}")
print(f"All shapefiles will be reprojected to: {target_crs}")

Reprojected shapefiles will be saved in: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected
All shapefiles will be reprojected to: EPSG:4326


In [ ]:
import glob
import geopandas as gpd
from pyproj import CRS

# Supported vector extensions
extensions = ['*.shp', '*.gpkg', '*.geojson', '*.json', '*.kml', '*.gml']
file_name_crs = target_crs.replace(":", "")

def reproject_layers(input_folder, output_folder, target_crs):
    files_to_process = []
    for ext in extensions:
        files_to_process.extend(glob.glob(os.path.join(input_folder, ext)))

    if not files_to_process:
        print("No supported vector files found in input folder.")
        return

    target_crs_obj = CRS.from_user_input(target_crs)

    for file_path in files_to_process:
        file_name = os.path.basename(file_path)
        try:
            gdf = gpd.read_file(file_path)
        except:
            print(f"Failed to read {file_name}")
            continue

        try:
            gdf = gdf.to_crs(target_crs_obj)
        except Exception as e:
            print(f"Failed to reproject {file_name}: {e}")
            continue

        base_name, ext = os.path.splitext(file_name)
        if ext.lower() == ".json":
            ext = ".geojson"
        output_file_name = f"{base_name}_{file_name_crs}{ext}"
        output_path = os.path.join(output_folder, output_file_name)

        try:
            gdf.to_file(output_path, driver="GPKG" if ext.lower()==".gpkg" else None)
            print(f"Reprojected layer saved to: {output_path}")
        except Exception as e:
            print(f"Error saving {file_name}: {e}")

reproject_layers(input_folder, output_folder, target_crs)
print("All layers reprojected!")

Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/NL_Province_Boundary_EPSG4326.gpkg
Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/NL_Municipal_Boundaries_EPSG4326.gpkg
Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/Newfoundland Boundary_EPSG4326.gpkg
Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/Labrador Boundary_EPSG4326.gpkg
Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/NL_Regions_EPSG4326.gpkg
Reprojected layer saved to: /content/drive/My Drive/Shapefiles/Shapefiles_Reprojected/NL_Dissolved_Regions_EPSG4326.gpkg
All layers reprojected!
